First we will load all voices into 5 sec chunks and label them healthy or unhealthy

In [ ]:
import os
import torch
import torchaudio
import torchaudio.transforms as T

# Paths
healthy_path = "data/healthy"
unhealthy_path = "data/unhealthy"


output_path = "processed_chunks"
os.makedirs(output_path, exist_ok=True)

chunk_duration = 5  # seconds
target_sr = 22050   # sampling rate

def process_folder(input_path, label):
    files = [f for f in os.listdir(input_path) if f.endswith(".wav")]
    print(f"Found {len(files)} files in {input_path}")

    all_chunks = []
    samples_per_chunk = target_sr * chunk_duration

    for file in files:
        file_path = os.path.join(input_path, file)

        # Load audio
        waveform, sr = torchaudio.load(file_path)

        # Convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Resample if needed
        if sr != target_sr:
            resampler = T.Resample(sr, target_sr)
            waveform = resampler(waveform)

        total_samples = waveform.shape[1]
        num_chunks = total_samples // samples_per_chunk
        remainder = total_samples % samples_per_chunk

        # Full 5s chunks
        for i in range(num_chunks):
            start = i * samples_per_chunk
            end = start + samples_per_chunk
            chunk = waveform[:, start:end]
            all_chunks.append((chunk, label))

            # Save chunk
            out_file = f"{os.path.splitext(file)[0]}_chunk{i}_{label}.wav"
            torchaudio.save(os.path.join(output_path, out_file), chunk, target_sr)

        # Handle remainder
        if remainder > 0:
            last_chunk = waveform[:, -remainder:]
            pad_size = samples_per_chunk - remainder
            padded = torch.nn.functional.pad(last_chunk, (0, pad_size))
            all_chunks.append((padded, label))

            out_file = f"{os.path.splitext(file)[0]}_lastchunk_{label}.wav"
            torchaudio.save(os.path.join(output_path, out_file), padded, target_sr)

    return all_chunks


# Process both sets
healthy_chunks = process_folder(healthy_path, label=0)     # 0 = healthy
unhealthy_chunks = process_folder(unhealthy_path, label=1) # 1 = unhealthy

# Combine dataset
dataset = healthy_chunks + unhealthy_chunks
print(f"Total chunks created: {len(dataset)}")


📂 Found 43 files in data/healthy


c:\Users\ayush\OneDrive\Desktop\Evrything\Workspace\Pandas\venv\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
c:\Users\ayush\OneDrive\Desktop\Evrything\Workspace\Pandas\venv\Lib\site-packages\torchaudio\_backend\utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: http

📂 Found 32 files in data/unhealthy
✅ Total chunks created: 2149


In [ ]:

from torch.utils.data import Dataset, DataLoader, random_split
import random


class VoiceDataset(Dataset):
    def __init__(self, chunk_dir, n_mels=64, augment=False):
        self.chunk_dir = chunk_dir
        self.files = [f for f in os.listdir(chunk_dir) if f.endswith(".wav")]
        self.augment = augment

        # Mel Spectrogram transform
        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=22050,
            n_fft=1024,
            hop_length=512,
            n_mels=n_mels
        )

    def add_augmentation(self, waveform, sr):
        if random.random() < 0.5:  # Add noise
            noise = torch.randn_like(waveform) * 0.005
            waveform = waveform + noise

        if random.random() < 0.5:  # Random gain (volume up/down)
            gain = random.uniform(0.8, 1.2)
            waveform = waveform * gain

        if random.random() < 0.3:  # Pitch shift
            n_steps = random.uniform(-2, 2)
            waveform = torchaudio.functional.pitch_shift(waveform, sr, n_steps)

        return waveform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        file_path = os.path.join(self.chunk_dir, file)

        # Load audio
        waveform, sr = torchaudio.load(file_path)

        # Convert to mono if needed
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Ensure consistent sample rate
        if sr != 22050:
            resampler = torchaudio.transforms.Resample(sr, 22050)
            waveform = resampler(waveform)

        # Apply augmentation only in training
        if self.augment:
            waveform = self.add_augmentation(waveform, 22050)

        # Convert to Mel spectrogram
        mel = self.mel_transform(waveform)
        mel_db = torchaudio.functional.amplitude_to_DB(
            mel, multiplier=10.0, amin=1e-10, db_multiplier=0
        )

        # Label from filename (_0 = healthy, _1 = unhealthy)
        label = 1 if "_1" in file else 0

        return mel_db, torch.tensor(label, dtype=torch.long)


# Create dataset
dataset = VoiceDataset("processed_chunks")

# Train-test split (80/20)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# ⚡ Enable augmentation only for train set
train_dataset.dataset.augment = True
test_dataset.dataset.augment = False

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

print(f"Total samples: {len(dataset)}")
print(f"Train: {len(train_dataset)}, Test: {len(test_dataset)}")



✅ Total samples: 2149
📊 Train: 1719, Test: 430


In [9]:
mel, label = dataset[0]
print("Mel shape:", mel.shape)
print("Label:", label)

Mel shape: torch.Size([1, 64, 216])
Label: tensor(0)


In [10]:
import torch.nn as nn
import torch.nn.functional as F


class CNNVoiceClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNVoiceClassifier, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.AdaptiveAvgPool2d((4, 4))  # compress to fixed size

        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNVoiceClassifier(num_classes=2).to(device)

weights = torch.tensor([1.0, 1.15]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for mel, labels in train_loader:
        mel, labels = mel.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(mel)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")


Epoch [1/20] Loss: 0.6260, Train Acc: 65.79%
Epoch [2/20] Loss: 0.5065, Train Acc: 74.40%
Epoch [3/20] Loss: 0.4392, Train Acc: 78.42%
Epoch [4/20] Loss: 0.3970, Train Acc: 80.57%
Epoch [5/20] Loss: 0.3460, Train Acc: 84.06%
Epoch [6/20] Loss: 0.3361, Train Acc: 84.06%
Epoch [7/20] Loss: 0.2960, Train Acc: 86.68%
Epoch [8/20] Loss: 0.2685, Train Acc: 88.02%
Epoch [9/20] Loss: 0.2609, Train Acc: 88.48%
Epoch [10/20] Loss: 0.2296, Train Acc: 89.53%
Epoch [11/20] Loss: 0.2215, Train Acc: 90.23%
Epoch [12/20] Loss: 0.1934, Train Acc: 91.16%
Epoch [13/20] Loss: 0.1908, Train Acc: 91.56%
Epoch [14/20] Loss: 0.1543, Train Acc: 93.66%
Epoch [15/20] Loss: 0.1629, Train Acc: 93.89%
Epoch [16/20] Loss: 0.1407, Train Acc: 93.83%
Epoch [17/20] Loss: 0.1297, Train Acc: 95.06%
Epoch [18/20] Loss: 0.1337, Train Acc: 93.83%
Epoch [19/20] Loss: 0.1047, Train Acc: 95.87%
Epoch [20/20] Loss: 0.1413, Train Acc: 94.42%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for mel, labels in test_loader:
        mel, labels = mel.to(device), labels.to(device)
        outputs = model(mel)
        probs = torch.softmax(outputs, dim=1)
        
        predicted = (probs[:, 1] > 0.45).long()
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

print("📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Healthy", "Unhealthy"]))

print("🧮 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


accuracy = 100. * np.sum(np.array(all_labels) == np.array(all_preds)) / len(all_labels)
print(f"✅ Overall Test Accuracy: {accuracy:.2f}%")

c:\Users\ayush\OneDrive\Desktop\Evrything\Workspace\Pandas\venv\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


📊 Classification Report:
              precision    recall  f1-score   support

     Healthy       0.95      0.88      0.92       258
   Unhealthy       0.84      0.94      0.88       172

    accuracy                           0.90       430
   macro avg       0.90      0.91      0.90       430
weighted avg       0.91      0.90      0.90       430

🧮 Confusion Matrix:
[[227  31]
 [ 11 161]]
✅ Overall Test Accuracy: 90.23%


In [20]:
torch.save(model.state_dict(), "cnn_voice_classifier_state.pth")